# **Please use @justinjohn colab notebook which is better and actually works  https://colab.research.google.com/github/justinjohn0306/Wav2Lip/blob/master/Wav2Lip_simplified_v5.ipynb**

# **There is no need for colab pro to use it.**
[Discord](https://discord.gg/F5WzXC8vXJ)




---



# **Get the code and models**

In [ ]:
#@title <h1>Step1: Setup Wav2Lip</h1>
#@markdown * Install dependency
#@markdown * Download pretrained model
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from IPython.display import clear_output
clear_output()
print("\nDone")


Done




---



# **Quick guide**
1. Create video file named input_video.mp4 - audio track removed
2. Create audio file named input_audio.wav
3. Both files have to be exact same length
4. Target face in the input_video.mp4, must be "detectable" in ALL videoframes (So no black or blurry frames etc)
5. Make sure u use correct file extensions
6. wav2lip does not like very long and high res clips (1080p/30seconds max)



# **Now lets try!**



---



In [ ]:
# #@title 1.Upload input_video.mp4 & input_audio.wav files
# %cd sample_data/
# from google.colab import files
# uploaded = files.upload()
# %cd ..

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
base_dir = '/content/drive/My Drive/tanishque/'
wav2lip_dir = '/content/Wav2Lip/'

In [ ]:
cd '/content/drive/My Drive/tanishque'

/content/drive/My Drive/tanishque


In [ ]:
import os
import ffmpeg

# Input and Output directories
input_dir = 'input'
seg_dir = 'segmented'

# Create the output directory if it doesn't exist
os.makedirs(seg_dir, exist_ok=True)

# Replace 'input_video.mp4' with the name of your uploaded video file in the 'input' folder
input_file = os.path.join(input_dir, 'input_video.mp4')

# Open the video file
probe = ffmpeg.probe(input_file)
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')

# Get the duration of the video in seconds
duration = float(video_info['duration'])

# Define the segment duration in seconds (20 seconds)
segment_duration = 20

# Calculate the number of segments
num_segments = int(duration / segment_duration)

# Lists to store segmented video and audio filenames
segmented_video_files = []
audio_files = []

for i in range(num_segments):
    start_time = i * segment_duration
    end_time = (i + 1) * segment_duration
    vid_output_file = os.path.join(seg_dir, f'vid_ns{i+1:04d}.mp4')
    aud_output_file = os.path.join(seg_dir, f'aud_{i+1:04d}.wav')  # Change the extension to .wav

    # Create the audio-less video
    (
        ffmpeg
        .input(input_file, ss=start_time, t=segment_duration)
        .output(vid_output_file, **{'c:v': 'copy', 'an': None})
        .run()
    )

    # Extract audio and save as a separate WAV file
    (
        ffmpeg
        .input(input_file, ss=start_time, t=segment_duration)
        .output(aud_output_file, **{'c:a': 'pcm_s16le', 'vn': None})
        .run()
    )

    # Append segmented video and audio filenames to lists
    segmented_video_files.append(vid_output_file)
    audio_files.append(aud_output_file)

Video successfully split into segments. Audio extracted in WAV format.


In [ ]:
# #@title 2.Create Wav2Lip video (using wav2lip_gan.pth) GAN
# !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav"

In [ ]:
cd '/content/Wav2Lip/'

/content/Wav2Lip


In [ ]:
import os
import subprocess
import platform

# Input and Output directories
base_dir = '/content/drive/My Drive/tanishque/'
wav2lip_dir = '/content/Wav2Lip/'
input_dir = os.path.join(base_dir, 'input')
seg_dir = os.path.join(base_dir, 'segmented')
processed_dir = os.path.join(base_dir, 'processed')

os.makedirs(processed_dir, exist_ok=True)

# List of segmented video filenames
segmented_video_files = [os.path.join(seg_dir, f'vid_ns{i+1:04d}.mp4') for i in range(num_segments)]

# Specify the command template with the --outfile argument

segmented_video_files = segmented_video_files[:3]
print(segmented_video_files)
# Iterate over segmented video files and run the command for each
for i, segmented_video_file in enumerate(segmented_video_files):
    input_audio = os.path.join(seg_dir, f'aud_{i+1:04d}.wav')
    output_file = os.path.join(processed_dir, f'result_voice_{i+1:04d}.mp4')

    # Build the command with the appropriate filenames using keyword arguments
    command = "python3 \"/content/Wav2Lip/inference.py\" --checkpoint_path \"/content/Wav2Lip/checkpoints/wav2lip_gan.pth\" --face " + "\"" + segmented_video_file + "\" --audio \"" + input_audio + "\" --outfile \"" + output_file + "\""

    # Run the command using subprocess
    print(command)
    subprocess.call(command, shell=platform.system() != 'Windows')

print("Inference completed on all segmented videos.")


['/content/drive/My Drive/tanishque/segmented/vid_ns0001.mp4', '/content/drive/My Drive/tanishque/segmented/vid_ns0002.mp4', '/content/drive/My Drive/tanishque/segmented/vid_ns0003.mp4']
python3 /content/Wav2Lip/inference.py --checkpoint_path "/content/Wav2Lip/checkpoints/wav2lip_gan.pth" --face "/content/drive/My Drive/tanishque/segmented/vid_ns0001.mp4" --audio "/content/drive/My Drive/tanishque/segmented/aud_0001.wav" --outfile "/content/drive/My Drive/tanishque/processed/result_voice_0001.mp4"
python3 /content/Wav2Lip/inference.py --checkpoint_path "/content/Wav2Lip/checkpoints/wav2lip_gan.pth" --face "/content/drive/My Drive/tanishque/segmented/vid_ns0002.mp4" --audio "/content/drive/My Drive/tanishque/segmented/aud_0002.wav" --outfile "/content/drive/My Drive/tanishque/processed/result_voice_0002.mp4"


KeyboardInterrupt: ignored

In [ ]:
command = "python3 inference.py --checkpoint_path \"checkpoints/wav2lip_gan.pth\" --face " + "\"" + segmented_video_file + "\" --audio \"" + input_audio + "\" --outfile \"" + output_file + "\""
print(command)

python3 inference.py --checkpoint_path "checkpoints/wav2lip_gan.pth" --face "/content/drive/My Drive/tanishque/segmented/vid_ns0003.mp4" --audio "/content/drive/My Drive/tanishque/segmented/aud_0003.wav" --outfile "/content/drive/My Drive/tanishque/processed/results/result_voice_0003.mp4"


In [ ]:
!python3 "/content/Wav2Lip/inference.py" --checkpoint_path "/content/Wav2Lip/checkpoints/wav2lip_gan.pth" --face "/content/drive/My Drive/tanishque/segmented/vid_ns0001.mp4" --audio "/content/drive/My Drive/tanishque/segmented/aud_0001.wav" --outfile "/content/drive/My Drive/tanishque/processed/result_voice_0001.mp4"

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 500
/content/Wav2Lip/audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
(80, 1601)
Length of mel chunks: 497
  0% 0/4 [00:00<?, ?it/s]
  0% 0/32 [00:00<?, ?it/s]
  3% 1/32 [00:34<18:03, 34.96s/it]
  6% 2/32 [00:37<07:58, 15.94s/it]
  9% 3/32 [00:39<04:42,  9.75s/it]
 12% 4/32 [00:42<03:11,  6.83s/it]
 16% 5/32 [00:44<02:20,  5.22s/it]
 19% 6/32 [00:47<01:51,  4.29s/it]
 22% 7/32 [00:49<01:33,  3.75s/it]
 25% 8/32 [00:52<01:19,  3.30s/it]
 28% 9/32 [00:54<01:09,  3.01s/it]
 31% 10/32 [00:56<01:01,  2.82s/it]
 34% 11/32 [00:59<00:56,  2.68s/it]
 38% 12/32 [01:01<00:53,  2.66s/it]
 41% 13/32 [01:04<00:50,  2.65s/it]
 44% 14/32 [01:06<00:46,  2.57s/it]
 47% 15/32 [01:09<00:42,  2.50s/it]
 50% 16/32 [01:11<00:40,  2.52s/it]
 53%

In [ ]:
#@title 3.Play result video -  50% scaling
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
#@title 4.Download Result.mp4 to your computer
from google.colab import files
files.download('/content/Wav2Lip/results/result_voice.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title 5. Delete old uploaded samples & result files, so you can start over again.
%rm /content/sample_data/*
%rm /content/Wav2Lip/results/*
from IPython.display import clear_output
clear_output()
print("\nDone! now press X")



---



In [ ]:
#@title 1-4. Batch processing - Upload -> process -> download -> play result
%cd sample_data/
%rm input_audio.wav
%rm input_video.mp4
from google.colab import files
uploaded = files.upload()
%cd ..
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav"
from google.colab import files
files.download('/content/Wav2Lip/results/result_voice.mp4')
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")



---



# **Variations to try**


In [ ]:
#@title 2.Use resize_factor to reduce the video resolution, as there is a change you might get better results for lower resolution videos. Why? Because the model was trained on low resolution faces.
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav" --resize_factor 2

In [ ]:
#@title 3.Use more padding to include the chin region (u can manually edit pads dimensions viewing and changing the code)
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav" --pads 0 20 0 0

In [ ]:
#@title 4.Play result video -  50% scaling
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
#@title 5.Download Result.mp4 to your computer
from google.colab import files
files.download('/content/Wav2Lip/results/result_voice.mp4')
